In [ ]:
# css_code.py and experiment.py combined

import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer # Corrected import for Aer
import random # Need to import random for apply_pauli_noise

# Standard [[15,7,3]] quantum Hamming code parity-check matrix (8×15)
H = np.array([
    [1,1,1,0,1,1,0,1,1,0,1,0,1,0,1],
    [1,0,1,1,1,0,1,1,0,1,0,1,0,1,0],
    [1,1,0,1,0,1,1,0,1,0,1,1,0,1,0],
    [0,1,1,1,1,1,1,1,0,0,1,1,1,0,0],
    [1,1,0,0,1,1,1,1,1,0,0,1,1,1,0],
    [1,0,1,1,1,1,1,0,1,1,0,0,1,1,1],
    [0,1,1,1,1,1,0,1,1,1,1,0,0,1,1],
    [1,0,1,1,1,0,1,1,1,1,1,1,0,0,1]
])  # Example: use any full-rank 8x15 binary matrix for Hamming code

num_physical = H.shape[1]    # 15
num_stab = H.shape[0]        # 8
num_logical = num_physical - num_stab  # 7

def prepare_logical_zero():
    qc = QuantumCircuit(num_physical)
    # Just |0>^15 for now—expand with encoding circuit if desired
    return qc

def apply_pauli_noise(qc, p):
    for q in range(num_physical):
        r = random.random()
        if r < 3*p:
            if r < p:
                qc.x(q)
            elif r < 2*p:
                qc.y(q)
            else:
                qc.z(q)
    return qc

def measure_stabilizers_XZ(qc, syndrome_bits):
    # Use 8 ancilla qubits, 8 classical bits for syndrome readout
    anc = QuantumRegister(num_stab, 'anc')
    cr = ClassicalRegister(num_stab, 's')
    qc.add_register(anc)
    qc.add_register(cr)
    # Each row of H defines a stabilizer
    for stab_idx in range(num_stab):
        for qubit in range(num_physical):
            if H[stab_idx, qubit]:
                qc.cx(qubit, anc[stab_idx])
        qc.measure(anc[stab_idx], cr[stab_idx])
    return qc

def decode_syndrome(syndrome):
    # Placeholder: always do nothing (identity)
    # For demonstration: you can implement a minimum-weight decoder or lookup table here
    return [None]*num_physical

def apply_recovery(qc, corrections):
    for idx, op in enumerate(corrections):
        if op == 'X':
            qc.x(idx)
        elif op == 'Y':
            qc.y(idx)
        elif op == 'Z':
            qc.z(idx)
    return qc

def readout_and_decode(qc):
    cr = ClassicalRegister(num_physical, 'm')
    qc.add_register(cr)
    qc.measure(list(range(num_physical)), cr)
    return qc

# experiment.py

# Try importing Aer; fall back to BasicAer if unavailable.
try:
    backend = Aer.get_backend('qasm_simulator')
except ImportError:
    from qiskit import BasicAer
    backend = BasicAer.get_backend('qasm_simulator')

# from qiskit import execute # This line is causing an error


def run_trial(p):
    """Runs a trial: state prep, noise, syndrome, recovery, measurement, decoding"""
    qc = prepare_logical_zero()
    qc = apply_pauli_noise(qc, p)
    qc = measure_stabilizers_XZ(qc, syndrome_bits=[0]*num_stab)
    corrections = decode_syndrome([0]*num_stab)
    qc = apply_recovery(qc, corrections)
    qc = readout_and_decode(qc)
    result = backend.run(qc, shots=1).result() # Use backend.run() instead of execute()
    outcome = list(result.get_counts().keys())[0]
    decoded = (outcome.count('1')) % 2 == 0  # placeholder logic, revise with real decoder
    return decoded

def sweep_p():
    ps = np.arange(0, 0.105, 0.005)
    n_trials = 200
    success = []
    for p in ps:
        succs = 0
        for _ in range(n_trials):
            succs += run_trial(p)
        success.append(succs / n_trials)
    np.savez('results_css.npz', ps=ps, success=success)
    print("Sweep complete and data saved.")

if __name__ == '__main__':
    sweep_p()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the results
data = np.load('results_css.npz')
ps = data['ps']
success = data['success']

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(ps, success, marker='o')
plt.xlabel('Pauli Error Probability (p)')
plt.ylabel('Logical Success Rate')
plt.title('Logical Success Rate vs. Pauli Error Probability')
plt.grid(True)
plt.show()